In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.decomposition import TruncatedSVD

stop_words = set(stopwords.words('english'))

FILE = '../data/canada/annoncexxx_17000_sample.csv'
df_orig = pd.read_csv(FILE)
content = []
# content = df_data['body']
# content = content.replace(np.nan, '', regex=True)
for index, row in df_orig.iterrows():
    c = ''
    # print (row['body'])
    # print (row['body'])
    # if type(row['title']) == type(' '):
    # 	c = c + row['title']
    if type(row['description']) == type(' '):
        c = c + row['description']
    # c = row['title'] + ' ' + row['body']
    c = re.sub(r'\d+', '', c)
#     if type(row['Name']) == type(''):
#         name = row['Name'].split(';')
#         for n in name:
#             name_regex = re.compile(re.escape(n), re.IGNORECASE)
#             c = name_regex.sub('', c)
    c = re.sub(r'[^\x00-\x7F]+',' ', c)
    cleanr = re.compile('<.*?>')
    c = re.sub(cleanr, '', c)
#     df_data.at[index, 'body'] = c
    content.append(c)
df_orig['content_p'] = content

vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(2,3), norm='l2', 
     stop_words=stop_words, min_df=2, max_df=0.8)
bigram_matrix = vectorizer.fit_transform(content)
features_col = vectorizer.get_feature_names()
print(bigram_matrix.shape)
# print (bigram_matrix[0])

# svd = TruncatedSVD(n_components=3)
# svd.fit_transform(bigram_matrix)
# np.save('../data/modalities_data/tf_idf_bigrams.npy', bigram_matrix)
print ("Done")

In [ ]:
svd = TruncatedSVD(n_components=20)
encoded_vecs = svd.fit_transform(bigram_matrix)

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, algorithm='best', alpha=1.0)
clusterer.fit(encoded_vecs)
#     print (clusterer.labels_)
labels = clusterer.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
unique_labels = set(labels)
probs = clusterer.probabilities_
#     colors = [plt.cm.Spectral(each)
#               for each in np.linspace(0, 1, len(set(true_labels))]
print ("Number of labels : " + str(len(list(set(clusterer.labels_)))))

In [ ]:
df_orig['cluster_label'] = labels
df_orig['sim_check'] = False
df_orig['sim_index'] = -1

In [ ]:
def calculate_pairwise_similarities(mat):
    mat = np.asarray(mat.todense())
    sim_scores = np.zeros((mat.shape[0], mat.shape[0]))
    for i in range(mat.shape[0]):
        for j in range(i+1, mat.shape[0]):
#             print ("i : {}, j : {}".format(i,j))
            if i == j:
                continue
#             print (len(mat[i]))
            sim_scores[i][j] = (mat[i]==mat[j]).all()
    return sim_scores

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

filtered_df = pd.DataFrame(columns=df_orig.columns)
# unique_labels = list(unique_labels) + [-1]
blacklisted_global_indices = []
for l in unique_labels:
    df_fil = df_orig[df_orig['cluster_label']==l].copy()
    if l == -1:
        filtered_df = pd.concat((filtered_df, df_fil), axis=0)
        continue
    indices = list(df_fil.index)
    df_fil.reset_index(drop=True, inplace=True)
    
    ads = list(df_fil['content_p'])
#     print (ads)
    count_vectorizer = CountVectorizer(lowercase=True, ngram_range=(2,3), stop_words=stop_words)
    ads_vectors = count_vectorizer.fit_transform(ads)
#     sim_scores = calculate_pairwise_similarities(ads_vectors)
    sim_scores = cosine_similarity(ads_vectors, dense_output=True)
    sim_scores *= np.tri(*sim_scores.shape)
    np.fill_diagonal(sim_scores, 0.0)

    indices_similar = np.where(sim_scores>0.998)
    cluster_tuples = zip(indices_similar[0], indices_similar[1])
#     print (cluster_tuples)
#     indices_similar = np.where(sim_scores == 1)
#     x = indices_similar[0]
#     y = indices_similar[1]
#     print (sim_scores)
#     print (y)
    blacklisted_indices = []
    
    for tup in cluster_tuples:
#         print (tup)
        if tup[0] > tup[1]:
            blacklisted_indices.append(indices[tup[1]])
            blacklisted_global_indices.append(indices[tup[1]])
#     print (df_orig.loc[blacklisted_global_indices, 'body'])
#     break
#     blacklisted_indices = list(set(blacklisted_indices))
    print (indices, set(blacklisted_indices))
    remain_ind = [x for x in indices if x not in blacklisted_indices]
    df_orig.loc[blacklisted_indices, 'sim_index'] = remain_ind[0] if len(remain_ind) >0 else -1
#     if len(df_fil) < 10:
#         print (df_fil['body'])
#         print (sim_scores)
#         print (blacklisted_indices)
#     df_fil.drop(blacklisted_indices, inplace=True)
#     filtered_df = pd.concat((filtered_df, df_fil), axis=0)
blacklisted_global_indices = list(set(blacklisted_global_indices))
df_orig.loc[blacklisted_global_indices, 'sim_check'] = True

#     break
print (df_orig.shape)

In [ ]:
print (df_orig.at[312, 'description'])
print (df_orig.at[406, 'description'])
print (df_orig.at[610, 'description'])
print (df_orig.at[1070, 'description'])

In [ ]:
print (len(set(blacklisted_global_indices)))

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.decomposition import TruncatedSVD

stop_words = set(stopwords.words('english'))

filtered_df = df_orig[df_orig['sim_check'] == False].copy()
print (filtered_df.shape)
content = []
# content = df_data['body']
# content = content.replace(np.nan, '', regex=True)
for index, row in filtered_df.iterrows():
    c = ''
    # print (row['body'])
    # print (row['body'])
    # if type(row['title']) == type(' '):
    # 	c = c + row['title']
    if type(row['description']) == type(' '):
        c = c + row['description']
    # c = row['title'] + ' ' + row['body']
    c = re.sub(r'\d+', '', c)
#     if type(row['Name']) == type(''):
#         name = row['Name'].split(';')
#         for n in name:
#             name_regex = re.compile(re.escape(n), re.IGNORECASE)
#             c = name_regex.sub('', c)
    c = re.sub(r'[^\x00-\x7F]+',' ', c)
    cleanr = re.compile('<.*?>')
    c = re.sub(cleanr, '', c)
#     df_data.at[index, 'body'] = c
    content.append(c)
filtered_df['content_p'] = content


vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(2,4), norm='l2', 
     stop_words=stop_words, min_df=2, max_df=0.8)
bigram_matrix = vectorizer.fit_transform(content)
features_col = vectorizer.get_feature_names()
print(bigram_matrix.shape)
# print (bigram_matrix[0])

# svd = TruncatedSVD(n_components=3)
# svd.fit_transform(bigram_matrix)
# np.save('../data/modalities_data/tf_idf_bigrams.npy', bigram_matrix)
print ("Done")

In [ ]:
s = bigram_matrix.sum(axis=1)
print (s.shape)
print (len(np.argwhere(s==0)))

In [ ]:
svd = TruncatedSVD(n_components=64)
encoded_vecs = svd.fit_transform(bigram_matrix)
# print (a.shape)

In [ ]:
print (encoded_vecs.shape)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
# rcParams['figure.figsize'] = 10,100
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
plt.imshow(encoded_vecs)
ax.set_aspect('equal')
plt.colorbar(orientation='vertical')
plt.show()

In [ ]:
from scipy.spatial import distance
origin = np.zeros(64)
dist = np.zeros(len(encoded_vecs), dtype=float)
for i in range(len(encoded_vecs)):
    dist[i] = distance.euclidean(origin, encoded_vecs[i])
    
noisy_set = np.argwhere(dist < 0.1)


In [ ]:
print(type(bigram_matrix))
print (bigram_matrix.shape)

In [ ]:
s = bigram_matrix.sum(axis=1)
print (s.shape)
print (len(np.argwhere(s==0)))

In [ ]:
filtered_df['noise'] = False
filtered_df['index1'] = filtered_df.index
filtered_df.reset_index(drop=True, inplace=True)
print (max(filtered_df.index))

In [ ]:
bigram_matrix = bigram_matrix.todense()

In [ ]:
from scipy.spatial import distance
# spatial.distance.euclidean(origin, encoded_vecs[0])
noisy_list = []
# filtered_df.reset_index(drop=True, inplace=True)
noisy_list=list(map(lambda x : x[0], noisy_set))
print(len(noisy_list))
filtered_df.loc[noisy_list, 'noise'] = True

# noisy_list

In [ ]:
filtered_df.set_index('index1', inplace=True)
df_orig = df_orig.join(filtered_df['noise'], how='outer')
filtered_df['index1'] = filtered_df.index
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
df_nonoise = filtered_df[filtered_df['noise'] == False].copy()
df_nonoise.reset_index(drop=True, inplace=True)
bigram_matrix = np.delete(bigram_matrix, noisy_list, axis=0)
encoded_vecs = np.delete(encoded_vecs, noisy_list, axis=0)

In [ ]:
print (encoded_vecs.shape)

In [ ]:
noisy_set = noisy_set.reshape(-1)

In [ ]:
import umap
X_embedded = umap.UMAP().fit_transform(encoded_vecs)

In [ ]:
print (encoded_vecs.shape)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 100,100
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
plt.imshow(encoded_vecs)
ax.set_aspect('equal')
plt.colorbar(orientation='vertical')
plt.show()

In [ ]:
# ######################### HBDSCAN ############################
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, algorithm='best', alpha=1.0)
clusterer.fit(encoded_vecs)
#     print (clusterer.labels_)
labels = clusterer.labels_
labels_clustering = labels
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
unique_labels = set(labels)
probs = clusterer.probabilities_
#     colors = [plt.cm.Spectral(each)
#               for each in np.linspace(0, 1, len(set(true_labels))]
print ("Number of labels : " + str(len(list(set(clusterer.labels_)))))
#     palette = sns.color_palette()
#     cluster_colors = [sns.desaturate(palette[col], sat)
#                       if col >= 0 else (0.5, 0.5, 0.5) for col, sat in
#                       zip(clusterer.labels_, clusterer.probabilities_)]

In [ ]:
# binary_true_labels = np.asarray(binary_true_labels)
# # noise_labels = np.where(labels==-1)
# no_noise_labels = np.where(labels!=-1)
# tr_labels = np.where(binary_true_labels==1)
# print (binary_true_labels.shape)
# print (no_noise_labels[0].shape)
# print (tr_labels)
# # lst3 = [value for value in tr_labels if value in noise_labels] 
# common_tp = np.intersect1d(no_noise_labels[0], tr_labels[0])
# print (common_tp.shape)
# total_p = len(common_tp)/len((tr_labels[0]))
# print (total_p)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
noise_labels = np.where(labels==-1)
# print (len(noise_labels[0]))
noise_vecs = encoded_vecs[noise_labels[0],:][:]
noise_dist = np.zeros(len(noise_vecs), dtype=float)
for i in range(len(noise_dist)):
    noise_dist[i] = distance.euclidean(origin, encoded_vecs[i])
# print (noise_dist)
noise_dist = np.sort(noise_dist)
plt.plot(noise_dist)
plt.show()

In [ ]:
# anomaly_indices = []
# rgba_colors = np.zeros((len(true_labels),4))
# for ind, cl in enumerate(labels):
#     if cl == -1:
#         continue
#     cluster_idx = np.argwhere(labels == cl).reshape(-1)
#     anomaly_indices += list(cluster_idx)
# rgba_colors[:, 0] = 0
# rgba_colors[:, 3] = 0.01
# # print (anomaly_indices)
# for ind in anomaly_indices:
#     rgba_colors[ind, 0] = 1
#     rgba_colors[ind, 3] = 0.1
    
# rcParams['figure.figsize'] = 20,10
# plt.scatter(X_embedded.T[0], X_embedded.T[1], color=rgba_colors)

In [ ]:

rcParams['figure.figsize'] = 10,10
plt.scatter(X_embedded.T[0], X_embedded.T[1], alpha=0.2)
# plt.title("Clustering accuracy={}, fmeasure_synth={}, number_of_labels={}".format(clustering_acc, fmeasure, 
#                                                                                   len(unique_labels)))
plt.show()

In [ ]:
rcParams['figure.figsize'] = 10,2
for i in range(10):
    ax1 = plt.subplot(1,5,1)
    ax2 = plt.subplot(1,5,2)
    ax3 = plt.subplot(1,5,3)
    ax4 = plt.subplot(1,5,4)
    ax5 = plt.subplot(1,5,5)
    
    ax1.scatter(encoded_vecs.T[i], encoded_vecs.T[i+1], alpha=0.1)
    ax1.title.set_text("{},{}".format(i, i+1))
    
    ax2.scatter(encoded_vecs.T[i], encoded_vecs.T[i+2], alpha=0.1)
    ax2.title.set_text("{},{}".format(i, i+2))
    
    ax3.scatter(encoded_vecs.T[i], encoded_vecs.T[i+3], alpha=0.1)
    ax3.title.set_text("{},{}".format(i, i+3))
    
    ax4.scatter(encoded_vecs.T[i], encoded_vecs.T[i+4], alpha=0.1)
    ax4.title.set_text("{},{}".format(i, i+4))
    
    ax5.scatter(encoded_vecs.T[i], encoded_vecs.T[i+5], alpha=0.1)
    ax5.title.set_text("{},{}".format(i, i+5))
    
    plt.tight_layout()
#     plt.savefig('../results/svd_components_{}.png'.format(i))
    plt.show()


In [ ]:
df_nonoise['label'] = -1 
val_counts = df_nonoise['phone'].value_counts()
unique_phone_numbers = val_counts.index.tolist()
print (len(unique_phone_numbers))
color_arr = []
for ind, u_p in enumerate(unique_phone_numbers):
    if val_counts[ind] > 1:
        df_nonoise.at[df_nonoise['phone'] == u_p, 'label'] = ind
#     print (df_data['label'].nunique())
print (df_nonoise['label'].nunique())

color_arr = []
df_labels = df_nonoise['label']
for l in df_labels:
    if l == -1:
        color_arr.append('grey')
    else:
        color_arr.append('red')

In [ ]:

rcParams['figure.figsize'] = 20,10
plt.scatter(X_embedded.T[0], X_embedded.T[1], c=color_arr, alpha=0.4)
# plt.title("Clustering accuracy={}, fmeasure_synth={}, number_of_labels={}".format(clustering_acc, fmeasure, 
#                                                                                   len(unique_labels)))
# plt.savefig('../results/embedding_canada_cluster_by_phone.png')
plt.show()

In [ ]:
rcParams['figure.figsize'] = 10,2


a = [0,1,2,3,4,5]
# b = [0,1,2,3,4,5]

from itertools import combinations 
tuples = list(combinations(a, 2))
count = 0
# for i in range(5):
#     if i%5 == 0:
#         plt.show()
#     else:
#         continue
#     ax1 = plt.subplot(1,5,1)
#     ax2 = plt.subplot(1,5,2)
#     ax3 = plt.subplot(1,5,3)
#     ax4 = plt.subplot(1,5,4)
#     ax5 = plt.subplot(1,5,5)
    
#     ax1.scatter(encoded_vecs.T[tuples[i][0]], encoded_vecs.T[tuples[i][1]], c=color_array, alpha=0.2)
#     ax1.title.set_text("{},{}".format(tuples[i][0], tuples[i][1]))
    
#     ax2.scatter(encoded_vecs.T[tuples[i+1][0]], encoded_vecs.T[tuples[i+1][1]], c=color_array, alpha=0.2)
#     ax2.title.set_text("{},{}".format(tuples[i+1][0], tuples[i+1][1]))
    
#     ax3.scatter(encoded_vecs.T[tuples[i+2][0]], encoded_vecs.T[tuples[i+2][1]], c=color_array, alpha=0.2)
#     ax3.title.set_text("{},{}".format(tuples[i+2][0], tuples[i+2][1]))
    
#     ax4.scatter(encoded_vecs.T[tuples[i+3][0]], encoded_vecs.T[tuples[i+3][1]], c=color_array, alpha=0.2)
#     ax4.title.set_text("{},{}".format(tuples[i+3][0], tuples[i+3][1]))
    
#     ax5.scatter(encoded_vecs.T[tuples[i+4][0]], encoded_vecs.T[tuples[i+4][1]], c=color_array, alpha=0.2)
#     ax5.title.set_text("{},{}".format(tuples[i+4][0], tuples[i+4][1]))
    
    
#     ax2.scatter(encoded_vecs.T[0], encoded_vecs.T[2], c=color_array, alpha=0.2)
#     ax2.title.set_text("{},{}".format(tup[i][0], tup[i][1]))
#     plt.tight_layout()
#     # plt.savefig('../results/svd_components_0.png')
#     plt.show()


for i in range(5):
    ax1 = plt.subplot(1,5,1)
    ax2 = plt.subplot(1,5,2)
    ax3 = plt.subplot(1,5,3)
    ax4 = plt.subplot(1,5,4)
    ax5 = plt.subplot(1,5,5)
    
    ax1.scatter(encoded_vecs.T[i], encoded_vecs.T[i+1], c=color_arr, alpha=0.2)
    ax1.title.set_text("{},{}".format(i, i+1))
    
    ax2.scatter(encoded_vecs.T[i], encoded_vecs.T[i+2], c=color_arr, alpha=0.2)
    ax2.title.set_text("{},{}".format(i, i+2))
    
    ax3.scatter(encoded_vecs.T[i], encoded_vecs.T[i+3], c=color_arr, alpha=0.2)
    ax3.title.set_text("{},{}".format(i, i+3))
    
    ax4.scatter(encoded_vecs.T[i], encoded_vecs.T[i+4], c=color_arr, alpha=0.2)
    ax4.title.set_text("{},{}".format(i, i+4))
    
    ax5.scatter(encoded_vecs.T[i], encoded_vecs.T[i+5], c=color_arr, alpha=0.2)
    ax5.title.set_text("{},{}".format(i, i+5))
    
    plt.tight_layout()
#     plt.savefig('../results/svd_components_{}.png'.format(i))
    plt.show()
#2,7
#1,6
#0,5
#4,9

In [ ]:
rcParams['figure.figsize'] = 8,2
ax1 = plt.subplot(1,4,1)
ax2 = plt.subplot(1,4,2)
ax3 = plt.subplot(1,4,3)
ax4 = plt.subplot(1,4,4)

ax1.scatter(encoded_vecs.T[2], encoded_vecs.T[7], c=color_arr, alpha=0.2)
ax1.text(0.5, 0.5, "{},{}".format(2, 7))
# ax1.title.set_text("{},{}".format(i, i+1))

ax2.scatter(encoded_vecs.T[1], encoded_vecs.T[6], c=color_arr, alpha=0.2)
ax2.text(0.5, 0.5,"{},{}".format(1, 6))

ax3.scatter(encoded_vecs.T[0], encoded_vecs.T[5], c=color_arr, alpha=0.2)
ax3.text(0.5, 0.5,"{},{}".format(0, 5))

ax4.scatter(encoded_vecs.T[4], encoded_vecs.T[9], c=color_arr, alpha=0.2)
ax4.text(0.5, 0.5,"{},{}".format(4, 9))

plt.tight_layout()
plt.savefig('../results/svd_components_to_use{}.png'.format(i))
plt.show()

In [ ]:
s = bigram_matrix.sum(axis=1)
print (s.shape)
print (len(np.argwhere(s==0)))

In [ ]:
def get_all_subgraphs(mat_data, cl_ind):
    filtered_sub = mat_data[cl_ind,:][:]
    bigrams_count = np.count_nonzero(filtered_sub, axis=0)
#     print ("Bigrams count matrix shape : {}".format(bigrams_count.shape))
    zero_count_index = np.where(bigrams_count==0)[1]
#     print (zero_count_index)
    one_count_index = list(np.where(bigrams_count==1)[1])
    core_bigrams_index = list(np.where(bigrams_count>1)[1])
    outer_bigrams_index = list(one_count_index + core_bigrams_index)
    
    bigram_induced_graph = mat_data[:][:,core_bigrams_index]
    ads_count = np.count_nonzero(bigram_induced_graph, axis=1)
    ads_in_shell_index = np.where(ads_count>1)[0]
    
    not_core_ads = list(set(list(ads_in_shell_index)) - set(cl_ind))
    not_core_bigrams = list(set(list(one_count_index)) - set(core_bigrams_index))
    mat_copy = np.asarray(mat_data)
    for i in not_core_ads:
        for j in not_core_bigrams:
            mat_copy[i][j] = 0
    shell_subgraph = mat_copy[ads_in_shell_index,:][:,outer_bigrams_index]
    mat_copy[not_core_ads, :] = 0
    outer_subgraph = mat_copy[ads_in_shell_index,:][:,outer_bigrams_index]
    mat_copy[:,one_count_index] = 0
    core_subgraph = mat_copy[ads_in_shell_index,:][:,outer_bigrams_index]
    shell_subgraph = np.asarray(shell_subgraph)
    outer_subgraph = np.asarray(outer_subgraph)
    core_subgraph = np.asarray(core_subgraph)

    return shell_subgraph, outer_subgraph, core_subgraph

In [ ]:
phone_counts = []
df_nonoise['cluster_label'] = labels
df_nonoise['probabilities'] = probs
for c in unique_labels:
    df_fil = df_nonoise[df_nonoise['cluster_label']==c]
    nun_phone = df_fil['phone'].nunique()
#     score = sum(df_fil['label'])/len(df_fil['label'])
    phone_counts.append(nun_phone)



In [ ]:
import math

def calculate_unweighted_density(core_mat):
    edge_weight = np.count_nonzero(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
#     print (ads_core_num)
#     print (bigrams_core_num)
    return edge_weight/(ads_core_num * bigrams_core_num + 1)

def calculate_weighted_density(core_mat):
    edge_weight = np.sum(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])

    return edge_weight/(ads_core_num * bigrams_core_num + 1)

def calculate_unweighted_fraudar_score(core_mat):
    edge_weight = np.count_nonzero(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return edge_weight/(ads_core_num + bigrams_core_num + 1)

def calculate_weighted_fraudar_score(core_mat):
    edge_weight = np.sum(core_mat)
    ads_count = np.count_nonzero(core_mat, axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    ads_core_num = len(np.where(ads_count>0)[0])
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return edge_weight/(ads_core_num + bigrams_core_num + 1)

def calculate_unweighted_edge_per_score(core_mat, outer_mat):
    core_edges = np.count_nonzero(core_mat)
    outer_edges = np.count_nonzero(outer_mat)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return ((core_edges + 1)/(outer_edges +1))*(math.log(bigrams_core_num+1))

def calculate_weighted_edge_per_score(core_mat, outer_mat):
    core_edges = np.sum(core_mat)
    outer_edges = np.sum(outer_mat)
    
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    
    return ((core_edges + 1)/(outer_edges +1))*(math.log(bigrams_core_num+1))


def calculate_custom_score(core_mat, outer_mat):
    bigram_degrees = np.count_nonzero(np.asarray(core_mat), axis=0)
    bigrams_count = np.count_nonzero(core_mat, axis=1)
    bigrams_core_num = len(np.where(bigrams_count>0)[0])
    outer_edges = np.count_nonzero(outer_mat)
    bigram_degrees = bigram_degrees/bigrams_core_num
    
    ads_count = np.count_nonzero(core_mat, axis=0)
    ads_core_num = len(np.where(ads_count>0)[0])
    
    return (np.sum(bigram_degrees)/(outer_edges+1))*(math.log(bigrams_core_num+1))*(math.log(ads_core_num+1))
# #     print (mat.shape)
#     edges_nonzero = np.count_nonzero(mat, axis=0)
#     unique, counts = np.unique(edges_nonzero, return_counts=True)
#     degree_counts = dict(zip(unique, counts))
#     numerator = 0.0
#     denominator = 0.0
#     half = max(mat.shape[0]/2, 2)
#     for k, v in degree_counts.items():
#         if k == 0:
#             continue
#         elif k <= half:
#             denominator += k*v
#         else:
#             denominator += k*v
#             numerator += k*v
#     if denominator == 0.0:
#         return 0.0
#     else:
#         return numerator/denominator

# def calculate_weighted_edge_per_score(mat):
#     return 0.0

#Should be shell_mat instead of outer_mat, change once you figure out how to get shell subgraph.
def calculate_unweighted_modularity_score(core_mat, outer_mat, total_edges):
#     total_edges = math.log(total_edges)
    ad_degrees = np.count_nonzero(np.asarray(outer_mat), axis=1)
#     print (ad_degrees)
    bigram_degrees = np.count_nonzero(np.asarray(outer_mat), axis=0)
#     print (bigram_degrees)
    ads_count = np.count_nonzero(core_mat, axis=1)
    bigrams_count = np.count_nonzero(core_mat, axis=0)
    ads_core_index = np.where(ads_count>0)[0]
    bigrams_core_index = np.where(bigrams_count>0)[0]
#     ads_list_index = [x for x in range(core_mat.shape[0])]
#     nonzero_edges = np.transpose(np.nonzero(core_mat))
    summation = 0.0
    for i in ads_core_index:
        for j in bigrams_core_index:
            if core_mat[i][j] == 0:
                adj = 0
            else:
                adj = 1
            if adj == 1:
                summation += (adj - (ad_degrees[i] * bigram_degrees[j])/total_edges)

#     nonzero_edges = np.transpose(np.nonzero(core_mat))
#     summation = 0.0
#     for k in range(nonzero_edges.shape[0]):
#         ad_index = nonzero_edges[k][0]
#         big_index = nonzero_edges[k][1]
# #         print (ad_index)
# #         print (big_index)
# #         if core_mat[ad_index][big_index] != 0:
# #             adj = 1
# #         else:
# #             adj = 0
        
#         summation += (1 - (ad_degrees[ad_index] * bigram_degrees[big_index])/(2*total_edges))
    
    return (summation/total_edges)*(math.log(len(ads_core_index)+1))

def calculate_weighted_modularity_score(core_mat, outer_mat, total_edges):
    ad_degrees = np.sum(np.asarray(outer_mat), axis=1)
    bigram_degrees = np.sum(np.asarray(outer_mat), axis=0)
    
    ads_count = np.count_nonzero(core_mat, axis=1)
    bigrams_count = np.count_nonzero(core_mat, axis=0)
    ads_core_index = np.where(ads_count>0)[0]
    bigrams_core_index = np.where(bigrams_count>0)[0]

    

#     ads_list_index = [x for x in range(core_mat.shape[0])]
#     nonzero_edges = np.transpose(np.nonzero(core_mat))
    summation = 0.0
    for i in ads_core_index:
        for j in bigrams_core_index:
            if core_mat[i][j] != 0:
                summation += (core_mat[i][j] - (ad_degrees[i] * bigram_degrees[j])/total_edges)
#     summation = 0.0
#     for k in range(nonzero_edges.shape[0]):
#         ad_index = nonzero_edges[k][0]
#         big_index = nonzero_edges[k][1]
#         summation += (outer_mat[ad_index][big_index] - (ad_degrees[ad_index] * bigram_degrees[big_index])/(2*total_edges))
    
    return (summation/total_edges)*(math.log(len(ads_core_index)+1))

def calculate_pairwise_modularity(mat):
    mat = np.asarray(mat.todense())
    sim_scores = np.zeros((mat.shape[0], mat.shape[0]))
    for i in range(mat.shape[0]):
        for j in range(i+1, mat.shape[0]):
#             print ("i : {}, j : {}".format(i,j))
            if i == j:
                continue
#             print (len(mat[i]))
            sim_scores[i][j] = calculate_modularity_score(np.vstack((mat[i], mat[j])))
    
    return sim_scores

In [ ]:
import gc
from sklearn.metrics.pairwise import cosine_similarity

def get_all_metrics(bigram_matrix, unique_labels, labels, df_data):
    eigen_ratios = []
    weighted_cluster_density = []
    unweighted_cluster_density = []
    unweighted_fraudar_scores = []
    weighted_fraudar_scores = []
    unweighted_outer_edge_perc_scores = []
    weighted_outer_edge_perc_scores = []
    unweighted_shell_edge_perc_scores = []
    weighted_shell_edge_perc_scores = []
    weighted_outer_modularity_scores = []
    unweighted_outer_modularity_scores = []
    weighted_shell_modularity_scores = []
    unweighted_shell_modularity_scores = []
    phone_counts = []
    pairwise_similarity = []
    custom_score = []
    clusters = []
    cluster_counts = []
    
    total_edges_unweighted = np.count_nonzero(bigram_matrix)
    total_edges_weighted = np.sum(bigram_matrix)
    for l in unique_labels:
#         s = bigram_matrix.sum(axis=1)
        if l== -1:
            weighted_cluster_density.append(0)
            unweighted_cluster_density.append(0)
            weighted_fraudar_scores.append(0)
            unweighted_fraudar_scores.append(0)
            weighted_outer_edge_perc_scores.append(0)
            unweighted_outer_edge_perc_scores.append(0)
            weighted_shell_edge_perc_scores.append(0)
            unweighted_shell_edge_perc_scores.append(0)
            unweighted_outer_modularity_scores.append(0)
            weighted_outer_modularity_scores.append(0)
            unweighted_shell_modularity_scores.append(0)
            weighted_shell_modularity_scores.append(0)
            pairwise_similarity.append(0)
            custom_score.append(0)

            cluster_counts.append(len(cluster_idx))
            
            
            eigen_ratios.append(0)
            clusters.append(l)
            phone_counts.append(0)
            
            continue
#         print (s.shape)
#         print ("bigram matrix sum : {}".format(bigram_matrix.sum()))
#         print ("Zero elems: {}".format(len(np.argwhere(s==0))))
        cluster_idx = np.argwhere(labels == l).reshape(-1)
#         print (l, len(cluster_idx))
        
        print (cluster_idx)
        shell_subgraph, outer_subgraph, core_subgraph = get_all_subgraphs(bigram_matrix.copy(), cluster_idx)
#         print (l, len(cluster_idx), core_subgraph.sum(), outer_subgraph.sum(), shell_subgraph.sum())
        

        df_filt = df_data[df_data['cluster_label']== l]
        if len(df_filt) == 0 or core_subgraph.shape[0] == 0 or shell_subgraph.shape[0] == 0 or outer_subgraph.shape[0] == 0:
            weighted_cluster_density.append(0)
            unweighted_cluster_density.append(0)
            weighted_fraudar_scores.append(0)
            unweighted_fraudar_scores.append(0)
            weighted_outer_edge_perc_scores.append(0)
            unweighted_outer_edge_perc_scores.append(0)
            weighted_shell_edge_perc_scores.append(0)
            unweighted_shell_edge_perc_scores.append(0)
            unweighted_outer_modularity_scores.append(0)
            weighted_outer_modularity_scores.append(0)
            unweighted_shell_modularity_scores.append(0)
            weighted_shell_modularity_scores.append(0)
            pairwise_similarity.append(0)
            custom_score.append(0)
            cluster_counts.append(len(cluster_idx))
            eigen_ratios.append(0)
            clusters.append(l)
            phone_counts.append(0)
            continue
#         elif core_subgraph.shape[0] == 0 or shell_subgraph.shape[0] == 0 or outer_subgraph.shape[0] == 0:
#             continue
        print (l, df_filt.shape)
        
        local_content = list(df_filt['content_p'])
        count_vectorizer = TfidfVectorizer(ngram_range=(2,2), use_idf=False)
        count_data = count_vectorizer.fit_transform(local_content)

        svd = TruncatedSVD(n_components=2)
        local_vecs = svd.fit_transform(count_data)
        w = svd.singular_values_
        eigen_rat = w[1]/w[0]
        eigen_ratios.append(eigen_rat)
        
        print (outer_subgraph.shape)
        print (core_subgraph.shape)
        pairwise_sim_mat = cosine_similarity(outer_subgraph, dense_output=True)
        pairwise_sim_mat = np.tril(pairwise_sim_mat, -1)
#         print (sum(pairwise_sim_mat).shape)
        print (pairwise_sim_mat.sum())
        an_score = calculate_weighted_edge_per_score(core_subgraph, outer_subgraph)
        phone = [x if type(x) == type('') else None for x in df_filt['phone'].unique()]
        if math.nan in phone:
            phone.remove(math.nan)
        if None in phone:
            phone.remove(None)
        phone = list(set(phone))
        c_score = an_score * max(0, len(phone)-1)
        print ("Scores : {}, {}, {}".format(an_score, len(phone), c_score))
        weighted_cluster_density.append(calculate_weighted_density(core_subgraph))
        unweighted_cluster_density.append(calculate_unweighted_density(core_subgraph))
        weighted_fraudar_scores.append(calculate_weighted_fraudar_score(core_subgraph))
        unweighted_fraudar_scores.append(calculate_unweighted_fraudar_score(core_subgraph))
        weighted_outer_edge_perc_scores.append(calculate_weighted_edge_per_score(core_subgraph, outer_subgraph))
        unweighted_outer_edge_perc_scores.append(calculate_unweighted_edge_per_score(core_subgraph, outer_subgraph))
        weighted_shell_edge_perc_scores.append(calculate_weighted_edge_per_score(core_subgraph, shell_subgraph))
        unweighted_shell_edge_perc_scores.append(calculate_unweighted_edge_per_score(core_subgraph, shell_subgraph))
        unweighted_outer_modularity_scores.append(calculate_unweighted_modularity_score(core_subgraph, outer_subgraph, total_edges_unweighted))
        weighted_outer_modularity_scores.append(calculate_weighted_modularity_score(core_subgraph, outer_subgraph, total_edges_weighted))
        unweighted_shell_modularity_scores.append(calculate_unweighted_modularity_score(core_subgraph, shell_subgraph, total_edges_unweighted))
        weighted_shell_modularity_scores.append(calculate_weighted_modularity_score(core_subgraph, shell_subgraph, total_edges_weighted))
        custom_score.append(c_score)
        phone_counts.append(len(phone))
        pairwise_similarity.append(pairwise_sim_mat.sum()/len(cluster_idx))
        cluster_counts.append(len(cluster_idx))
        clusters.append(l)
        
        count_data = []
        local_content = []
        shell_subgraph = []
        core_subgraph = []
        outer_subgraph = []
        if l % 50 == 0:
            print (l)
            gc.collect()
#     original_labels = labels.copy()

    metrics = {}
    metrics['weighted_cluster_density'] = weighted_cluster_density
    metrics['unweighted_cluster_density'] = unweighted_cluster_density
    metrics['weighted_fraudar_scores'] = weighted_fraudar_scores
    metrics['unweighted_fraudar_scores'] = unweighted_fraudar_scores
    metrics['weighted_outer_edge_perc_scores'] = weighted_outer_edge_perc_scores
    metrics['unweighted_outer_edge_perc_scores'] = unweighted_outer_edge_perc_scores
    metrics['weighted_shell_edge_perc_scores'] = weighted_shell_edge_perc_scores
    metrics['unweighted_shell_edge_perc_scores'] = unweighted_shell_edge_perc_scores
    metrics['unweighted_outer_modularity_scores'] = unweighted_outer_modularity_scores
    metrics['weighted_outer_modularity_scores'] = weighted_outer_modularity_scores
    metrics['unweighted_shell_modularity_scores'] = unweighted_shell_modularity_scores
    metrics['weighted_shell_modularity_scores'] = weighted_shell_modularity_scores
    metrics['pairwise_similarity'] = pairwise_similarity
    metrics['custom_score'] = custom_score
    metrics['phone_counts'] = phone_counts
    metrics['eigen_ratios'] = eigen_ratios
    metrics['clusters'] = clusters
    metrics['labels'] = labels.copy()
    metrics['cluster_counts'] = cluster_counts
    
    return metrics


In [ ]:
# import gc

# def get_all_metrics(bigram_matrix, unique_labels, labels, df_data):
#     eigen_ratios = []
#     weighted_cluster_density = []
#     unweighted_cluster_density = []
#     unweighted_fraudar_scores = []
#     weighted_fraudar_scores = []
#     unweighted_outer_edge_perc_scores = []
#     weighted_outer_edge_perc_scores = []
#     unweighted_shell_edge_perc_scores = []
#     weighted_shell_edge_perc_scores = []
#     weighted_outer_modularity_scores = []
#     unweighted_outer_modularity_scores = []
#     weighted_shell_modularity_scores = []
#     unweighted_shell_modularity_scores = []
#     clusters = []
#     cluster_counts = []
    
#     total_edges_unweighted = np.count_nonzero(bigram_matrix)
#     total_edges_weighted = np.sum(bigram_matrix)
#     for l in unique_labels:
# #         s = bigram_matrix.sum(axis=1)
#         if l== -1:
#             weighted_cluster_density.append(0)
#             unweighted_cluster_density.append(0)
#             weighted_fraudar_scores.append(0)
#             unweighted_fraudar_scores.append(0)
#             weighted_outer_edge_perc_scores.append(0)
#             unweighted_outer_edge_perc_scores.append(0)
#             weighted_shell_edge_perc_scores.append(0)
#             unweighted_shell_edge_perc_scores.append(0)
#             unweighted_outer_modularity_scores.append(0)
#             weighted_outer_modularity_scores.append(0)
#             unweighted_shell_modularity_scores.append(0)
#             weighted_shell_modularity_scores.append(0)
#             cluster_counts.append(math.log(len(cluster_idx), 10)) 
#             eigen_ratios.append(0)
#             clusters.append(l)

#             continue
# #         print (s.shape)
# #         print ("bigram matrix sum : {}".format(bigram_matrix.sum()))
# #         print ("Zero elems: {}".format(len(np.argwhere(s==0))))
#         cluster_idx = np.argwhere(labels == l).reshape(-1)
# #         print (l, len(cluster_idx))
#         clusters.append(l)
#         shell_subgraph, outer_subgraph, core_subgraph = get_all_subgraphs(bigram_matrix.copy(), cluster_idx)
# #         print (l, len(cluster_idx), core_subgraph.sum(), outer_subgraph.sum(), shell_subgraph.sum())
        

#         df_filt = df_data[df_data['cluster_label']== l]
        
#         local_content = list(df_filt['content_p'])
#         try:
#             count_vectorizer = TfidfVectorizer(ngram_range=(2,2), use_idf=False)
#             count_data = count_vectorizer.fit_transform(local_content)
#             if count_data.shape[1] < 5:
#                 eigen_ratios.append(1)
#             else:
#                 svd = TruncatedSVD(n_components=2)
#                 local_vecs = svd.fit_transform(count_data)
#                 w = svd.singular_values_
#                 eigen_rat = w[1]/w[0]
#                 eigen_ratios.append(eigen_rat)
#         except:
#             eigen_ratios.append(1)

#         weighted_cluster_density.append(calculate_weighted_density(core_subgraph))
#         unweighted_cluster_density.append(calculate_unweighted_density(core_subgraph))
#         weighted_fraudar_scores.append(calculate_weighted_fraudar_score(core_subgraph))
#         unweighted_fraudar_scores.append(calculate_unweighted_fraudar_score(core_subgraph))
#         weighted_outer_edge_perc_scores.append(calculate_weighted_edge_per_score(core_subgraph, outer_subgraph))
#         unweighted_outer_edge_perc_scores.append(calculate_unweighted_edge_per_score(core_subgraph, outer_subgraph))
#         weighted_shell_edge_perc_scores.append(calculate_weighted_edge_per_score(core_subgraph, shell_subgraph))
#         unweighted_shell_edge_perc_scores.append(calculate_unweighted_edge_per_score(core_subgraph, shell_subgraph))
#         unweighted_outer_modularity_scores.append(calculate_unweighted_modularity_score(core_subgraph, outer_subgraph, total_edges_unweighted))
#         weighted_outer_modularity_scores.append(calculate_weighted_modularity_score(core_subgraph, outer_subgraph, total_edges_weighted))
#         unweighted_shell_modularity_scores.append(calculate_unweighted_modularity_score(core_subgraph, shell_subgraph, total_edges_unweighted))
#         weighted_shell_modularity_scores.append(calculate_weighted_modularity_score(core_subgraph, shell_subgraph, total_edges_weighted))
#         cluster_counts.append(math.log(len(cluster_idx)+1, 10)) 
        
#         count_data = []
#         local_content = []
#         shell_subgraph = []
#         core_subgraph = []
#         outer_subgraph = []
#         if l % 50 == 0:
#             print (l)
#             gc.collect()
# #     original_labels = labels.copy()

#     metrics = {}
#     metrics['weighted_cluster_density'] = weighted_cluster_density
#     metrics['unweighted_cluster_density'] = unweighted_cluster_density
#     metrics['weighted_fraudar_scores'] = weighted_fraudar_scores
#     metrics['unweighted_fraudar_scores'] = unweighted_fraudar_scores
#     metrics['weighted_outer_edge_perc_scores'] = weighted_outer_edge_perc_scores
#     metrics['unweighted_outer_edge_perc_scores'] = unweighted_outer_edge_perc_scores
#     metrics['weighted_shell_edge_perc_scores'] = weighted_shell_edge_perc_scores
#     metrics['unweighted_shell_edge_perc_scores'] = unweighted_shell_edge_perc_scores
#     metrics['unweighted_outer_modularity_scores'] = unweighted_outer_modularity_scores
#     metrics['weighted_outer_modularity_scores'] = weighted_outer_modularity_scores
#     metrics['unweighted_shell_modularity_scores'] = unweighted_shell_modularity_scores
#     metrics['weighted_shell_modularity_scores'] = weighted_shell_modularity_scores
#     metrics['eigen_ratios'] = eigen_ratios
#     metrics['clusters'] = clusters
#     metrics['labels'] = labels.copy()
#     metrics['cluster_counts'] = cluster_counts
    
#     return metrics


In [ ]:
pre_merging_metrics = get_all_metrics(bigram_matrix, unique_labels, labels, df_nonoise)

In [ ]:
import gc
gc.collect()

In [ ]:
rcParams['figure.figsize'] = 5,5
clusters = pre_merging_metrics['clusters']
pre_merging_metrics['eigen_ratios'][pre_merging_metrics['clusters'].index(-1)] = 1


In [ ]:
eigen_ratios = pre_merging_metrics['eigen_ratios']
eigen_np = np.array(eigen_ratios)
clusters_nonhomogenous_index = np.where(eigen_np > 0.8)[0]
clusters_nonhomogenous = [clusters[i] for i in clusters_nonhomogenous_index]
# print (clusters_nonhomogenous)
non_noisy_clusters = [x for x in clusters if x not in clusters_nonhomogenous]

In [ ]:
rerun_cluster_idx = []
original_labels = pre_merging_metrics['labels'].copy()
for l in clusters_nonhomogenous:
    cluster_idx = np.argwhere(original_labels == l).reshape(-1)
    rerun_cluster_idx += list(cluster_idx)
#     print (total_cluster_idx)
print (len(rerun_cluster_idx))
rerun_bigram_data = bigram_matrix[rerun_cluster_idx, :]

In [ ]:
svd = TruncatedSVD(n_components=64)
encoded_vecs_rerun = svd.fit_transform(rerun_bigram_data)

In [ ]:
clusterer.fit(encoded_vecs_rerun)
#     print (clusterer.labels_)
rerun_clustering_labels = clusterer.labels_
rerun_labels_clustering = labels
rerun_n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_rerun_ = list(labels).count(-1)
rerun_unique_labels = set(labels)
rerun_probs = clusterer.probabilities_
#     colors = [plt.cm.Spectral(each)
#               for each in np.linspace(0, 1, len(set(true_labels))]
print ("Number of labels : " + str(len(list(set(clusterer.labels_)))))
#     palette = sns.color_palette()

In [ ]:
max_clusters = max(pre_merging_metrics['clusters'])
# labels = pre_merging_metrics['labels']
rerun_clusters = []
for ind, ad in enumerate(rerun_cluster_idx):
    if rerun_clustering_labels[ind] != -1:
        original_labels[ad] = rerun_clustering_labels[ind] + max_clusters + 1
        rerun_clusters.append(rerun_clustering_labels[ind] + max_clusters + 1)
        probs[ad] = rerun_probs[ind]
    else:
        original_labels[ad] = -1
        probs[ad] = rerun_probs[ind]
        rerun_clusters.append(-1)
rerun_clusters = list(set(rerun_clusters))
clusters = rerun_clusters + non_noisy_clusters
# print (clusters)

In [ ]:

post_splitting_metrics = get_all_metrics(bigram_matrix, rerun_clusters, original_labels, df_nonoise)

In [ ]:
eigen_ratios = []
weighted_cluster_density = []
unweighted_cluster_density = []
unweighted_fraudar_scores = []
weighted_fraudar_scores = []
unweighted_outer_edge_perc_scores = []
weighted_outer_edge_perc_scores = []
unweighted_shell_edge_perc_scores = []
weighted_shell_edge_perc_scores = []
weighted_outer_modularity_scores = []
unweighted_outer_modularity_scores = []
weighted_shell_modularity_scores = []
unweighted_shell_modularity_scores = []
pairwise_similarity = []
custom_score = []
phone_counts = []
cluster_counts = []
print (len(rerun_clusters))
print (len(post_splitting_metrics['eigen_ratios']))
for i in clusters:
    if i in rerun_clusters:
        ind = rerun_clusters.index(i)
        eigen_ratios.append(post_splitting_metrics['eigen_ratios'][ind])
        weighted_cluster_density.append(post_splitting_metrics['weighted_cluster_density'][ind])
        unweighted_cluster_density.append(post_splitting_metrics['unweighted_cluster_density'][ind])
        unweighted_fraudar_scores.append(post_splitting_metrics['unweighted_fraudar_scores'][ind])
        weighted_fraudar_scores.append(post_splitting_metrics['weighted_fraudar_scores'][ind])
        unweighted_outer_edge_perc_scores.append(post_splitting_metrics['unweighted_outer_edge_perc_scores'][ind])
        weighted_outer_edge_perc_scores.append(post_splitting_metrics['weighted_outer_edge_perc_scores'][ind])
        unweighted_shell_edge_perc_scores.append(post_splitting_metrics['unweighted_shell_edge_perc_scores'][ind])
        weighted_shell_edge_perc_scores.append(post_splitting_metrics['weighted_shell_edge_perc_scores'][ind])
        weighted_outer_modularity_scores.append(post_splitting_metrics['weighted_outer_modularity_scores'][ind])
        unweighted_outer_modularity_scores.append(post_splitting_metrics['unweighted_outer_modularity_scores'][ind])
        weighted_shell_modularity_scores.append(post_splitting_metrics['weighted_shell_modularity_scores'][ind])
        unweighted_shell_modularity_scores.append(post_splitting_metrics['unweighted_shell_modularity_scores'][ind])
        pairwise_similarity.append(post_splitting_metrics['pairwise_similarity'][ind])
        custom_score.append(post_splitting_metrics['custom_score'][ind])
        phone_counts.append(post_splitting_metrics['phone_counts'][ind])
        cluster_counts.append(post_splitting_metrics['cluster_counts'][ind])
        
    elif i in non_noisy_clusters:
        ind = pre_merging_metrics['clusters'].index(i)
        eigen_ratios.append(pre_merging_metrics['eigen_ratios'][ind])
        weighted_cluster_density.append(pre_merging_metrics['weighted_cluster_density'][ind])
        unweighted_cluster_density.append(pre_merging_metrics['unweighted_cluster_density'][ind])
        unweighted_fraudar_scores.append(pre_merging_metrics['unweighted_fraudar_scores'][ind])
        weighted_fraudar_scores.append(pre_merging_metrics['weighted_fraudar_scores'][ind])
        unweighted_outer_edge_perc_scores.append(pre_merging_metrics['unweighted_outer_edge_perc_scores'][ind])
        weighted_outer_edge_perc_scores.append(pre_merging_metrics['weighted_outer_edge_perc_scores'][ind])
        unweighted_shell_edge_perc_scores.append(pre_merging_metrics['unweighted_shell_edge_perc_scores'][ind])
        weighted_shell_edge_perc_scores.append(pre_merging_metrics['weighted_shell_edge_perc_scores'][ind])
        weighted_outer_modularity_scores.append(pre_merging_metrics['weighted_outer_modularity_scores'][ind])
        unweighted_outer_modularity_scores.append(pre_merging_metrics['unweighted_outer_modularity_scores'][ind])
        weighted_shell_modularity_scores.append(pre_merging_metrics['weighted_shell_modularity_scores'][ind])
        unweighted_shell_modularity_scores.append(pre_merging_metrics['unweighted_shell_modularity_scores'][ind])
        pairwise_similarity.append(pre_merging_metrics['pairwise_similarity'][ind])
        custom_score.append(pre_merging_metrics['custom_score'][ind])
        phone_counts.append(pre_merging_metrics['phone_counts'][ind])
        cluster_counts.append(pre_merging_metrics['cluster_counts'][i])

metrics = {}
metrics['weighted_cluster_density'] = weighted_cluster_density
metrics['unweighted_cluster_density'] = unweighted_cluster_density
metrics['weighted_fraudar_scores'] = weighted_fraudar_scores
metrics['unweighted_fraudar_scores'] = unweighted_fraudar_scores
metrics['weighted_outer_edge_perc_scores'] = weighted_outer_edge_perc_scores
metrics['unweighted_outer_edge_perc_scores'] = unweighted_outer_edge_perc_scores
metrics['weighted_shell_edge_perc_scores'] = weighted_shell_edge_perc_scores
metrics['unweighted_shell_edge_perc_scores'] = unweighted_shell_edge_perc_scores
metrics['unweighted_outer_modularity_scores'] = unweighted_outer_modularity_scores
metrics['weighted_outer_modularity_scores'] = weighted_outer_modularity_scores
metrics['unweighted_shell_modularity_scores'] = unweighted_shell_modularity_scores
metrics['weighted_shell_modularity_scores'] = weighted_shell_modularity_scores
metrics['pairwise_similarity'] = pairwise_similarity
metrics['custom_score'] = custom_score
metrics['phone_counts'] = phone_counts
metrics['eigen_ratios'] = eigen_ratios
metrics['clusters'] = clusters
metrics['cluster_label'] = labels.copy()
metrics['cluster_counts'] = cluster_counts 

post_noisy_split_metrics = metrics
print (len(post_noisy_split_metrics['eigen_ratios']))
print (len(clusters))

In [ ]:
cluster_labels_results = df_nonoise['cluster_label']
cluster_vectors = np.zeros((len(clusters), bigram_matrix.shape[1]))
for ind, cl in enumerate(clusters):
    cluster_idx = np.argwhere(cluster_labels_results == cl).reshape(-1)
    tf_cluster_mat = bigram_matrix[cluster_idx,:][:]
    tf_cluster_mat_flat = np.mean(tf_cluster_mat, axis=0)
    print (tf_cluster_mat_flat.shape)
    cluster_vectors[ind] = tf_cluster_mat_flat

where_are_nans = np.isnan(cluster_vectors)
cluster_vectors[where_are_nans] = 0
pairwise_sim_mat = cosine_similarity(cluster_vectors, dense_output=True)

In [ ]:
pairwise_sim_mat *= np.tri(*pairwise_sim_mat.shape)
np.fill_diagonal(pairwise_sim_mat, 0.0)

sim_clusters = np.where(pairwise_sim_mat>0.8)
cluster_tuples = zip(sim_clusters[0], sim_clusters[1])

clusters_to_be_merged = []

for tup in cluster_tuples:
    clusters_to_be_merged.append((clusters[tup[0]], clusters[tup[1]]))
print (clusters_to_be_merged)

In [ ]:
pre_clusters = np.array(cluster_labels_results)
print (bigram_matrix.shape)
for cl in clusters_to_be_merged:
    np_clusters = np.array(cluster_labels_results)
#     adj_matrix_copy = bigram_matrix.copy()
    replace_index = np.where(np_clusters==cl[1])
    for i in replace_index:
        np_clusters[i] = cl[0]
    pre_cl_ind1 = np.argwhere(pre_clusters==cl[0]).reshape(-1)
    pre_cl_ind2 = np.argwhere(pre_clusters==cl[1]).reshape(-1)
    print (pre_cl_ind1)
    pre_shell1, pre_outer1, pre_core1 = get_all_subgraphs(bigram_matrix.copy(), pre_cl_ind1)
    pre_shell2, pre_outer2, pre_core2 = get_all_subgraphs(bigram_matrix.copy(), pre_cl_ind2)
    total_edges = np.sum(bigram_matrix)
    mod1 = calculate_weighted_edge_per_score(pre_core1, pre_outer1)
    mod2 = calculate_weighted_edge_per_score(pre_core2, pre_outer2)
    print (np.sum(pre_shell1))
    post_cl_ind = np.argwhere(np_clusters==cl[0]).reshape(-1)
    post_shell, post_outer, post_core = get_all_subgraphs(bigram_matrix.copy(), post_cl_ind)
    post_mod = calculate_weighted_edge_per_score(post_core, post_outer)
    
    print ("Cluster1: {}, Cluster2: {}, Pre mod1: {}, Pre Mod2: {}, Post Mod: {}".format(cl[0], cl[1], mod1, mod2, post_mod))

In [ ]:
# pre_clusters = np.array(cluster_labels_results)
# print (bigram_matrix.shape)
# for cl in clusters_to_be_merged:
#     np_clusters = np.array(cluster_labels_results)
# #     adj_matrix_copy = bigram_matrix.copy()
#     replace_index = np.where(np_clusters==cl[1])
#     for i in replace_index:
#         np_clusters[i] = cl[0]
#     pre_cl_ind1 = np.argwhere(pre_clusters==cl[0]).reshape(-1)
#     pre_cl_ind2 = np.argwhere(pre_clusters==cl[1]).reshape(-1)
# #     print (pre_cl_ind1)
#     pre_shell1, pre_outer1, pre_core1 = get_all_subgraphs(bigram_matrix.copy(), pre_cl_ind1)
#     pre_shell2, pre_outer2, pre_core2 = get_all_subgraphs(bigram_matrix.copy(), pre_cl_ind2)
#     total_edges = np.sum(bigram_matrix)
#     mod1 = calculate_weighted_edge_per_score(pre_core1, pre_outer1)
#     mod2 = calculate_weighted_edge_per_score(pre_core2, pre_outer2)
# #     print (np.sum(pre_shell1))
#     post_cl_ind = np.argwhere(np_clusters==cl[0]).reshape(-1)
#     post_shell, post_outer, post_core = get_all_subgraphs(bigram_matrix.copy(), post_cl_ind)
#     post_mod = calculate_weighted_edge_per_score(post_core, post_outer)
    
# #     print ("Cluster1: {}, Cluster2: {}, Pre mod1: {}, Pre Mod2: {}, Post Mod: {}".format(cl[0], cl[1], mod1, mod2, post_mod))

In [ ]:
eigen_ratios = post_noisy_split_metrics['eigen_ratios']
weighted_cluster_density = post_noisy_split_metrics['weighted_cluster_density']
unweighted_cluster_density = post_noisy_split_metrics['unweighted_cluster_density']
unweighted_fraudar_scores = post_noisy_split_metrics['unweighted_fraudar_scores']
weighted_fraudar_scores = post_noisy_split_metrics['weighted_fraudar_scores']
unweighted_outer_edge_perc_scores = post_noisy_split_metrics['unweighted_outer_edge_perc_scores']
weighted_outer_edge_perc_scores = post_noisy_split_metrics['weighted_outer_edge_perc_scores']
unweighted_shell_edge_perc_scores = post_noisy_split_metrics['unweighted_shell_edge_perc_scores']
weighted_shell_edge_perc_scores = post_noisy_split_metrics['weighted_shell_edge_perc_scores']
weighted_outer_modularity_scores = post_noisy_split_metrics['weighted_outer_modularity_scores']
unweighted_outer_modularity_scores = post_noisy_split_metrics['unweighted_outer_modularity_scores']
weighted_shell_modularity_scores = post_noisy_split_metrics['weighted_shell_modularity_scores']
unweighted_shell_modularity_scores = post_noisy_split_metrics['unweighted_shell_modularity_scores']
pairwise_similarity = post_noisy_split_metrics['pairwise_similarity']
custom_score = post_noisy_split_metrics['custom_score']
phone_counts = post_noisy_split_metrics['phone_counts']
cluster_counts = post_noisy_split_metrics['cluster_counts']
labels = df_nonoise['cluster_label']
df_a = df_nonoise.copy()
# labels = df_data['cluster_label']
to_calculate_clusters = []
for tup in clusters_to_be_merged:
    if tup[1] in clusters:
        df_nonoise['cluster_label'].replace(tup[1], tup[0], inplace=True)
    # Handle case when 3 clusters are similar to each other eg. (a,b) (c,b)
#         print (df_a['cluster_label'].unique())

        ind = clusters.index(tup[1])
        del eigen_ratios[ind]
        del weighted_cluster_density[ind]
        del unweighted_cluster_density[ind]
        del unweighted_fraudar_scores[ind]
        del weighted_fraudar_scores[ind]
        del unweighted_outer_edge_perc_scores[ind]
        del weighted_outer_edge_perc_scores[ind]
        del unweighted_shell_edge_perc_scores[ind]
        del weighted_shell_edge_perc_scores[ind]
        del weighted_outer_modularity_scores[ind]
        del unweighted_outer_modularity_scores[ind]
        del weighted_shell_modularity_scores[ind]
        del unweighted_shell_modularity_scores[ind]
        del pairwise_similarity[ind]
        del custom_score[ind]
        del phone_counts[ind]
        del cluster_counts[ind]
        del clusters[ind]
        if tup[1] in to_calculate_clusters:
            to_calculate_clusters.remove(tup[1])
        to_calculate_clusters.append(tup[0])
    else:
        continue

metrics = get_all_metrics(bigram_matrix, to_calculate_clusters, df_nonoise['cluster_label'], df_nonoise)
for i, cl in enumerate(to_calculate_clusters):
    ind = clusters.index(cl)
    eigen_ratios[ind] = metrics['eigen_ratios'][i]
    weighted_cluster_density[ind] = post_noisy_split_metrics['weighted_cluster_density'][i]
    unweighted_cluster_density[ind] = post_noisy_split_metrics['unweighted_cluster_density'][i]
    unweighted_fraudar_scores[ind] = post_noisy_split_metrics['unweighted_fraudar_scores'][i]
    weighted_fraudar_scores[ind] = post_noisy_split_metrics['weighted_fraudar_scores'][i]
    unweighted_outer_edge_perc_scores[ind] = post_noisy_split_metrics['unweighted_outer_edge_perc_scores'][i]
    weighted_outer_edge_perc_scores[ind] = post_noisy_split_metrics['weighted_outer_edge_perc_scores'][i]
    unweighted_shell_edge_perc_scores[ind] = post_noisy_split_metrics['unweighted_shell_edge_perc_scores'][i]
    weighted_shell_edge_perc_scores[ind] = post_noisy_split_metrics['weighted_shell_edge_perc_scores'][i]
    weighted_outer_modularity_scores[ind] = post_noisy_split_metrics['weighted_outer_modularity_scores'][i]
    unweighted_outer_modularity_scores[ind] = post_noisy_split_metrics['unweighted_outer_modularity_scores'][i]
    weighted_shell_modularity_scores[ind] = post_noisy_split_metrics['weighted_shell_modularity_scores'][i]
    unweighted_shell_modularity_scores[ind] = post_noisy_split_metrics['unweighted_shell_modularity_scores'][i]
    pairwise_similarity[ind] = post_noisy_split_metrics['pairwise_similarity'][i]
    custom_score[ind] = post_noisy_split_metrics['custom_score'][i]
    phone_counts[ind] = post_noisy_split_metrics['phone_counts'][i]
    cluster_counts[ind] = post_noisy_split_metrics['cluster_counts'][i]

post_merging_metric = {}
post_merging_metric['weighted_cluster_density'] = weighted_cluster_density
post_merging_metric['unweighted_cluster_density'] = unweighted_cluster_density
post_merging_metric['weighted_fraudar_scores'] = weighted_fraudar_scores
post_merging_metric['unweighted_fraudar_scores'] = unweighted_fraudar_scores
post_merging_metric['weighted_outer_edge_perc_scores'] = weighted_outer_edge_perc_scores
post_merging_metric['unweighted_outer_edge_perc_scores'] = unweighted_outer_edge_perc_scores
post_merging_metric['weighted_shell_edge_perc_scores'] = weighted_shell_edge_perc_scores
post_merging_metric['unweighted_shell_edge_perc_scores'] = unweighted_shell_edge_perc_scores
post_merging_metric['unweighted_outer_modularity_scores'] = unweighted_outer_modularity_scores
post_merging_metric['weighted_outer_modularity_scores'] = weighted_outer_modularity_scores
post_merging_metric['unweighted_shell_modularity_scores'] = unweighted_shell_modularity_scores
post_merging_metric['weighted_shell_modularity_scores'] = weighted_shell_modularity_scores
post_merging_metric['pairwise_similarity'] = pairwise_similarity
post_merging_metric['custom_score'] = custom_score
post_merging_metric['phone_counts'] = phone_counts
post_merging_metric['eigen_ratios'] = eigen_ratios
post_merging_metric['clusters'] = clusters
post_merging_metric['cluster_label'] = df_nonoise['cluster_label'].copy()
post_merging_metric['cluster_counts'] = cluster_counts 

In [ ]:
print (len(post_merging_metric['clusters']), df_nonoise['cluster_label'].nunique(), len(post_merging_metric['eigen_ratios']))

In [ ]:
df_nonoise.set_index('index1', inplace=True)

In [ ]:
# df_a = df_orig.copy()
df_nonoise['final_label'] = df_nonoise['cluster_label']
df_orig = df_orig.join(df_nonoise['final_label'], how='outer')



In [ ]:
df_orig['final_label'].isna().sum()



In [ ]:
print (df_orig[df_orig['noise'] == True].shape)


In [ ]:
for ind, row in df_orig.iterrows():
    if row['noise'] == True:
        df_orig.at[ind, 'final_label'] = -2
    elif row['sim_check'] == True:
        df_orig.at[ind, 'final_label'] = df_orig.at[row['sim_index'], 'final_label']
    

In [ ]:
# for cl in clusters:
#     df_filt = df_orig[df_orig['final_label'] == cl]
#     names = [x.lower() if type(x) == type('') else None for x in df_filt['Name'].unique()]
#     if math.nan in names:
#         names.remove(math.nan)
#     if None in names:
#         names.remove(None)
#     names = list(set(names))
#     print (cl, len(names))
# # print (post_merging_metric['cluster_counts'][post_merging_metric['custom_score'].index(max(post_merging_metric['custom_score']))])

In [ ]:
suspicious_scores = post_merging_metric['custom_score']
text_similarity = post_merging_metric['pairwise_similarity']
clusters = post_merging_metric['clusters']
phone_counts = post_merging_metric['phone_counts']
w_density = post_merging_metric['weighted_cluster_density']
uw_density = post_merging_metric['unweighted_cluster_density']
w_fraudar = post_merging_metric['weighted_fraudar_scores']
uw_fraudar = post_merging_metric['unweighted_fraudar_scores']
w_outer_edge = post_merging_metric['weighted_outer_edge_perc_scores']
uw_outer_edge = post_merging_metric['unweighted_outer_edge_perc_scores']
w_shell_edge = post_merging_metric['weighted_shell_edge_perc_scores']
uw_shell_edge = post_merging_metric['unweighted_shell_edge_perc_scores']
w_outer_mod = post_merging_metric['weighted_outer_modularity_scores']
uw_outer_mod = post_merging_metric['unweighted_outer_modularity_scores']
w_shell_mod = post_merging_metric['weighted_shell_modularity_scores']
uw_shell_mod = post_merging_metric['unweighted_shell_modularity_scores']
cluster_counts = post_merging_metric['cluster_counts']

In [ ]:
import seaborn as sns


ax = sns.regplot(x=phone_counts, y=suspicious_scores, x_ci=68, truncate=False, lowess=True, scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Unique phone numbers per cluster', ylabel='anomaly score')
plt.show()

ax = sns.regplot(x=phone_counts, y=w_outer_edge, x_ci=68, truncate=False, lowess=True, scatter_kws = {'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Unique phone numbers per cluster', ylabel='Weighted outer edge percentage')
plt.show()

# import seaborn as sns
ax = sns.regplot(x=text_similarity, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Text Similarity', ylabel='Weighted outer edge percentage')
plt.show()


ax = sns.regplot(x=phone_counts, y=text_similarity, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
ax.set(xlabel='Unique phone numbers per cluster', ylabel='Text Similarity')
plt.show()


ax = sns.scatterplot(x=cluster_counts, y=phone_counts, ci=68, alpha=0.4)
ax.set(xlabel='Number of ads', ylabel='Unique phone numbers per cluster')
plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()

# import seaborn as sns
# ax = sns.regplot(x=avg_label_scores, y=w_outer_edge, ci=68, truncate=False, lowess=True, scatter_kws = {'color': 'b', 'alpha': 0.2}, line_kws = {'color': 'red'})
# ax.set(xlabel='Average Label Score', ylabel='Weighted outer edge percentage')
# plt.show()


# plt.scatter(avg_label_scores, w_outer_edge, alpha=0.2)
# plt.title('Avg label score vs weighted outer edge percentage')
# plt.xlabel('avg label score')
# plt.ylabel('weighted outer edge percentage')
# # w = np.linalg.lstsq(avg_label_scores, w_outer_edge)[0]
# # yh = np.dot(avg_label_scores,w)
# # plt.plot(avg_label_scores, yh, 'r-')
# plt.show()

# plt.scatter(avg_label_scores, w_shell_edge, alpha=0.2)
# plt.title('Avg label score vs weighted shell edge percentage')
# plt.xlabel('avg label score')
# plt.ylabel('weighted shell edge percentage')
# plt.show()

# plt.scatter(text_similarity, w_outer_edge, alpha=0.2)
# plt.title('Avg text similarity vs weighted outer edge percentage')
# plt.xlabel('avg text similarity')
# plt.ylabel('weighted outer edge percentage')

# # plt.scatter(avg_label_scores, w_shell_edge, alpha=0.2)
# # plt.title('Avg label score vs weighted shell edge percentage')
# # plt.xlabel('avg label score')
# # plt.ylabel('weighted shell edge percentage')
# plt.show()

In [ ]:
ind = [i for i, e in enumerate(cluster_counts) if e> 100]
print(ind)

In [ ]:
clusters[67]

In [ ]:
df_filt = df_orig[df_orig['cluster_label'] == 69]

for ind, row in df_filt.iterrows():
    print ('------------------------------------------------------------')
    print (row['description'], row['phone'])